In [180]:
"""
@author: ChewingGumKing_OJF
"""
import json
import os
import sys
import time
from dataclasses import dataclass
from datetime import datetime
#loads necessary libraries
from typing import Union, Any, Tuple, Sequence, List
import re
from bs4 import BeautifulSoup as bs

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import warnings
import requests
from GlobalVariable import *
from GlobalFunctions import GlobalFunctions

error: str = ''
options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

#options.add_argument("--headless")

path = GlobalVariable.ChromeDriverPath
driver = webdriver.Chrome(options=options, executable_path=path)

def date_transformation(date: str) -> Tuple[str, str]:
    match = re.search(r'(\d{1,2})-(\d{1,2})\s*(\w+)\s*(\d{4})', date)
    if match:
        change = lambda exact: datetime.strptime(exact, '%d %B %Y').strftime('%Y-%m-%d')
        return tuple(map(lambda no: change(' '.join(match.group(no, *(3, 4)))), [1, 2]))

#*******************************************************************************************************************
import logging
def creating_log(script_name: str):

    log_folder_path = 'log_folder'

    if os.path.exists(log_folder_path):
        for files in os.listdir(log_folder_path):
            if files == 'jupyter_run.log':
                os.remove(os.path.join(os.getcwd(), log_folder_path, files))
    else:
        os.makedirs(log_folder_path)

    log_path = os.path.join(os.getcwd(), log_folder_path, f'{script_name}.log')

    logger = logging.getLogger(script_name)
    logger.setLevel(logging.DEBUG)
    log_handler = logging.FileHandler(log_path)
    log_format = logging.Formatter(
        '%(asctime)s -- %(name)s -- %(levelname)s -- %(message)s \n\n')
    log_handler.setFormatter(log_format)
    logger.addHandler(log_handler)
    logger.info('Log reporting is instantiated.')

    return logger


logger = creating_log('jupyter_run')
#*******************************************************************************************************************

/tmp/ipykernel_12619/4042217731.py:36: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=path)


In [181]:
def date_tran(m, d, y):
    str_t = ' '.join((d,m,y))
    return datetime.strptime(str_t, '%d %b %Y').strftime('%Y-%m-%d')

def get_date_range(date_str):
    if date_str == '':
        return ('', '')
    date_match = re.search(r'([A-Za-z]{3})\s*(\d{0,2})\s*(?:\s*,\s*(\d{4})\s*)?.+?([A-Za-z]{3})?\s*(\d{0,2})\s*,.+?(\d{4})', date_str)
    date_g = date_match.groups()
    s_m, s_d, s_y, e_m, e_d, e_y = date_g
    if e_m == None:
        e_m = s_m
    if s_y == None:
        s_y = e_y
    prev = date_tran(s_m, s_d, s_y)
    end = date_tran(e_m, e_d, e_y)
    return (prev, end)

def get_price(price_s):
    if price_s == '':
        return '[]'
    p_s = price_s.split(' - ')
    return json.dumps(
        list(map(lambda x: {'type': 'paid', 'price': x[1:], 'currency': x[0]}, p_s)),
        ensure_ascii=False
    )

def f_time(time_s):
    if time_s == '':
        return '[]'
    t_r = re.search(r'(\d{1,2}(?:.\d{1,2})?\s*[AaMPm]{2})\s*.+?\s*(\d{1,2}(?:.\d{1,2})?\s*[aMPAm]{2})\s*([a-zA-Z]{2,3})', time_s)
    t_s, t_e, t_z = tuple(map(lambda x: x.replace(' ', ''), t_r.groups()))
    return json.dumps([{'type':'paid', 'start_time':t_s, 'end_time':t_e, 'timezone':t_z, 'days':'all'}], ensure_ascii = False)


def manipVals(val):
    val[1] = get_date_range(val[1])
    val[2] = get_price(val[2])
    val[3] = f_time(val[3])
    return val

class ScrapeEvent:
    """ the codebase design uses a Class with it Methods as function scraping singular data(some more,
        in the case of going inside the page just once). It returns the data to a it caller which is handled by a context manager.
    """

    browser: WebDriver = driver
    wait_5sec: WebDriverWait = WebDriverWait(browser, 5)
    error_msg_from_class: str = ''

    def __enter__(self):
        "Handles the contex manager."
        return self

    def __exit__(self, exc_type=None, exc_value=None, exc_tb=None):
        self.browser.quit()

    def get_link(self, link: str) -> None:
        "Handles the teardown of the context manager."
        self.browser.get(link)

    def dispatch(self, locator:str, strategy:webdriver = By.CSS_SELECTOR):
        "API calls for Selenium webdriver.find_element"
        return self.browser.find_element(strategy, locator)

    def dispatchList(self, locator:str, strategy: webdriver = By.CSS_SELECTOR):
        "API calls for Selenium webdriver.find_elements"
        return self.browser.find_elements(strategy, locator)

    def get_done(self) -> Any:
            "Scrapes and return event venue "
            try:
                page_prop = self.browser.execute_script("return "\
                "$('.details').map(function() {"\
                "child_list = $(this).children();"\
                "child_length = child_list.length;"\
                "info = $('.article-detail p').first().text();"\
                "switch (child_length) {"\
                "    case 3 :{"\
                "        date_str = '';"\
                "        price_str = $(child_list[1]).text();"\
                "        time_str = '';"\
                "        virtual = 1;"\
                "        venue = '';"\
                "        city = '';"\
                "        state = '';"\
                "        break;"\
                "    }"\
                "    case 5 :{"\
                "        date_str = $(child_list[0]).text();"\
                "        price_str = $(child_list[3]).text();"\
                "        time_str = $($(child_list[1]).children()[1]).html();"\
                "        virtual = 1;"\
                "        venue = '';"\
                "        city = '';"\
                "        state = '';"\
                "        break;"\
                "    }"\
                "    default :{"\
                "        date_str = $(child_list[0]).text();"\
                "        price_str = $(child_list[4]).text();"\
                "        time_str = $($(child_list[1]).children()[1]).html();"\
                "        virtual = 0;"\
                "        venue = $(child_list[3]).text();"\
                "        metro = $(child_list[2]).text().split(' , ');"\
                "        city = metro[0];"\
                "        state = metro[1];"\
                "    }"\
                "}"\
                "return [[info, date_str, price_str, time_str, virtual, venue, city, state]];"\
                "});"
                )
                page_prop = map(manipVals, page_prop)
            except Exception as e:
                self.error_msg_from_class += '\n' + str(e)
                logger.error(f'{self.get_events.__name__} Function failed', exc_info=True)
            else:
                return list(page_prop)



In [182]:
url = 'https://www.theiia.org/en/products/learning-solutions/on-demand/auditing-anti-corruption/'

In [183]:
with ScrapeEvent() as handler:
    handler.browser.implicitly_wait(5)

    try:
        links = handler.get_link(url)
    except Exception as e:
        logger.error('Connection Error', exc_info=True)
        error += str(e) 
        
    try:
        print('-----********************-----')
        res = handler.get_done()
        print(res)
    except Exception as e:
        logger.error('Function failed', exc_info=True)
        error +=  str(e)

    error += handler.error_msg_from_class


print('-----********************-----')
print('DONE')


-----********************-----
[['', ('', ''), '[{"type": "paid", "price": "59", "currency": "$"}]', '[]', 1, '', '', '']]
-----********************-----
DONE


In [184]:
string ='22-23 november 2022'
match = re.search(r'(\d{1,2})-(\d{1,2})\s*(\w+)\s*(\d{4})', string)
if match:
    change = lambda exact: datetime.strptime(exact, '%d %B %Y').strftime('%Y-%m-%d')
    match_dat =map(lambda no: change(' '.join(match.group(no, *(3, 4)))), [1, 2])

print(list(match_dat))


['2022-11-22', '2022-11-23']
